# DASC521: Homework 2
## Sina Dadmand


## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import math

# Reading and dividing dataset:

In [2]:
images = pd.read_csv('hw02_images.csv', header = None)
labels = pd.read_csv('hw02_labels.csv', header = None)

train = images.loc[:29999,]
test = images.loc[30000:,]

train_y = labels.loc[:29999, 0]
test_y = labels.loc[30000:, 0]

# Parameter estimations for the training dataset


In [3]:
# get number of classes and number of samples
K = int(np.max(labels))
N = train.shape[0]

# calculate sample means and deviations
sample_means = [train[train_y == (c + 1)].apply(lambda x: np.array(x.mean())).astype(np.float64) for c in range(K)]
sample_deviations  = [train[train_y == (c + 1)].apply(lambda x: np.array(x.std(ddof=0))).astype(np.float64) for c in range(K)]
class_priors = [len(train_y[train_y == (c+1)]) / len(train_y) for c in range(K)]

print(sample_means)
print(sample_deviations)
print(class_priors)

[0      254.998667
1      254.984167
2      254.856167
3      254.667333
4      254.544667
          ...    
779    253.132167
780    254.236500
781    254.679000
782    254.878167
783    254.959333
Length: 784, dtype: float64, 0      254.997333
1      254.997333
2      254.996500
3      254.994167
4      254.870500
          ...    
779    254.582333
780    254.904500
781    254.968833
782    254.992167
783    254.988667
Length: 784, dtype: float64, 0      254.999333
1      254.999333
2      254.992333
3      254.976500
4      254.879667
          ...    
779    199.764167
780    233.056333
781    251.524833
782    254.472500
783    254.974833
Length: 784, dtype: float64, 0      254.996667
1      254.989833
2      254.914167
3      254.692167
4      254.189167
          ...    
779    229.081500
780    242.632333
781    252.395167
782    254.441667
783    254.936667
Length: 784, dtype: float64, 0      254.999000
1      254.984333
2      254.937833
3      254.772500
4      254.497000
 

# Score function for Bayes classifier and max score finder

In [7]:
tiny = 1e-308 # for the safelog

# Naive-Bayes 
def bayes(x, mean, dev, prior):
    P = (1/(dev * np.sqrt(2 * np.pi))) * np.exp(((mean - x)*(x - mean))/(2 * dev * dev))
    P = np.log(P+tiny)
    return np.sum(P) + np.log(prior)



In [8]:
train_pred = np.empty(shape=(len(train_y),))

for i in range(len(train_y)):
    train_pred[i] = np.argmax(np.array([bayes(train.loc[i,:], sample_means[0], sample_deviations[0], class_priors[0]), 
                                          bayes(train.loc[i,:], sample_means[1], sample_deviations[1], class_priors[1]),
                                          bayes(train.loc[i,:], sample_means[2], sample_deviations[2], class_priors[2]),
                                          bayes(train.loc[i,:], sample_means[3], sample_deviations[3], class_priors[3]),
                                          bayes(train.loc[i,:], sample_means[4], sample_deviations[4], class_priors[4]),
                                         ])) + 1

In [9]:
test_pred = np.empty(shape=(len(test_y),))

for i in range(len(test_y)):
    test_pred[i] = np.argmax(np.array([bayes(test.loc[N+i,:], sample_means[0], sample_deviations[0], class_priors[0]), 
                                         bayes(test.loc[N+i,:], sample_means[1], sample_deviations[1], class_priors[1]),
                                         bayes(test.loc[N+i,:], sample_means[2], sample_deviations[2], class_priors[2]),
                                         bayes(test.loc[N+i,:], sample_means[3], sample_deviations[3], class_priors[3]),
                                         bayes(test.loc[N+i,:], sample_means[4], sample_deviations[4], class_priors[4]),
                                         ])) + 1




# Confusion matrix

In [11]:
confusion_matrix_train = pd.crosstab(train_pred, train_y, rownames = ['y_pred'], colnames = ['y_truth'])
confusion_matrix_test = pd.crosstab(test_pred, test_y, rownames = ['y_pred'], colnames = ['y_truth'])
print(confusion_matrix_train)
print(confusion_matrix_test)

y_truth     1     2     3     4     5
y_pred                               
1.0      3685    49     4   679     6
2.0      1430  5667  1140  1380   532
3.0       508   208  4670  2948   893
4.0       233    59   123   686   180
5.0       144    17    63   307  4389
y_truth    1    2    3    4    5
y_pred                          
1.0      597    6    0  114    1
2.0      237  955  188  267   81
3.0       92   25  785  462  167
4.0       34   11   16  109   29
5.0       40    3   11   48  722
